<a href="https://colab.research.google.com/github/ajakrish/Ajay_info5502_spring2022/blob/main/INFO_5502_Final_Exam_Q3_Spring2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style='background:yellow'>**Question 3 (100 pt, each sub-question is 10 pt).**</span> 

Part A: Given Inetd dataset, an intrusion detection dataset that contains 2 columns. The first column represents a process ID number (PID) and the second column represents a system call. The goal is to extract the entire system call sequence by its PID, then tokenize and parse each sequence into 25-grams. After this, we’ll split the data into training and testing sets. Next, we ensure data quality in the training dataset by removing duplicate sequences within each class and overlapped sequences between both classes. To ensure a fair evaluation, we also clean the testing set by repeating the previous step. You will implement a Decision Tree model to train and test on the training set and testing set that we just clean. 


Part B: As the data type is sequential, we want to know how similar the sequences from both classes are and visualize how the similarity scores vary in different sequence lengths. Compute a cosine similarity score of 100 random pairs from both classes and get a median cosine similarity. Repeat this step in different sequence lengths (5, 10, 15, and 20). Plot a line chart to visualize how the median cosine similarities vary through sequence lengths of 5 to 25. 

You can download the datasets from github: https://github.com/unt-iialab/info5502-spring2022/tree/main/finalexam/q4


Step by step instructions and questions:

(1) Attached are the Normal and Intrusion data folders. Read in the data and store them in two separate dataframes. Combine the system calls (the second column) from the same PID (the first column) into a sequence or a string. 

(2) Parse an entire sequence into smaller sequences of size 25. (Hint: Use word_tokenize and ngrams from nltk library to parse).

(3) Add labels to the data, and partition the labeled data into training data and testing data with a ratio of 70/30.

(4) As the dataset has a lot of duplicate sequences, make sure that the testing data only contain unique sequences and that these sequences do not exist in the training data. Condense training data and testing data into two sets. Take the difference between the testing set and training set to remove the overlapped sequences from the testing data. 

(5) The sequences from each class can be repetitive, and some of these sequences may have different labels. Therefore, we need to remove the duplicate sequences within each class and the overlapped sequences from the Intrusion class. Start out by separating the Normal class and Intrusion class in the training data. Use the same concept in Question 4 to remove duplicate sequences and overlapped sequences from the training data.

(6) Repeat Question 5 to clean the testing data.

(7) Bootstrap the training data to get the same number of sequences from each class. Train and evaluate a Decision Tree model using the training data and testing data that we just clean. How is the performance?

(8) Randomly sample 100 pairs of sequences from both classes and calculate the median cosine similarity.

(9) Refer back to Question 2, parse the data into a length of 5, 10, 15, and 20. Clean the data from each class using Question 5. For each sequence length, calculate the median cosine similarity of 100 random pairs of sequences from both classes. 

(10) Plot a line chart to visualize how the median cosine similarities vary through different sequence lengths from 5 to 25.


In [1]:
#### You code here
# 1.)
import pandas as pd
import numpy as np

intrusion = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int", sep = " ", header = None)
normal = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int", sep = " ", header = None)

intrusion.columns = ['PID', 'SystemCalls']
normal.columns = ['PID', 'SystemCalls']

print(intrusion.head(2))
print()
print(normal.head(2))

   PID  SystemCalls
0  167           90
1  167          125

   PID  SystemCalls
0  167           90
1  167          125


In [2]:
print(intrusion.shape)
print(normal.shape)

(8371, 2)
(541, 2)


In [3]:
idict = {}
ndict = {}

# combining system calls
for pid in list(set(intrusion['PID'].values)):
    idict[pid] = " ".join(str(value) for value in list(intrusion[intrusion['PID'] == pid]['SystemCalls'].values))

for pid in list(set(normal['PID'].values)):
    ndict[pid] = " ".join(str(value) for value in list(normal[normal['PID'] == pid]['SystemCalls'].values))

idict[167]

'90 125 125 106 5 90 6 5 3 90 90 90 90 6 125 91 125 125 125 136 49 24 47 50 45 45 5 106 106 106 106 106 2 1'

In [4]:
import re
from nltk.util import ngrams
ilist = []
nlist = []

#generating ngrams with sequence length of 25

for key in idict.keys():
    s = idict[key]
    tokens = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokens, 25)):
        ilist.append(ele)

for key in ndict.keys():
    s = ndict[key]
    tokens = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokens, 25)):
        nlist.append(ele)

print(len(ilist))
print(len(nlist))

7627
469


In [5]:
idf = pd.DataFrame(ilist)
ndf = pd.DataFrame(nlist)

# adding labels to intrusion and normal data sequences
idf['class'] = 1
ndf['class'] = 0

print(idf.shape)
print(ndf.shape)

(7627, 26)
(469, 26)


In [6]:
combined_df1 = pd.DataFrame()

combined_df1 = combined_df1.append(idf)
combined_df1 = combined_df1.append(ndf)

print(combined_df1.shape)

(8096, 26)


In [7]:
#Removing duplicates
combined_df1 = combined_df1.drop_duplicates(subset = combined_df1.columns[:-1], keep = 'last')
combined_df1.shape

(333, 26)

In [8]:
combined_df1['class'].value_counts()

1    177
0    156
Name: class, dtype: int64

In [9]:
combined_df1.columns[:-1]

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24],
      dtype='object')

In [10]:
#decision tree classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X = combined_df1.drop("class", axis = 1)
y = combined_df1['class']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

dt_classifier = DecisionTreeClassifier(max_depth = 10)
dt_classifier.fit(X_train, y_train)
y_predict = dt_classifier.predict(X_valid)

print(classification_report(y_valid, y_predict))


              precision    recall  f1-score   support

           0       0.75      0.82      0.78        22
           1       0.91      0.87      0.89        45

    accuracy                           0.85        67
   macro avg       0.83      0.84      0.83        67
weighted avg       0.86      0.85      0.85        67



In [11]:
'''
Similarity scores for NGram range of 5 to 25
'''

'\nSimilarity scores for NGram range of 5 to 25\n'

In [12]:
import re
from nltk.util import ngrams
import pandas as pd

cosine_similar_ngram = {}

for ngram in range(5, 26, 5):

    intrusion = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int", sep = " ", header = None)
    normal = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int", sep = " ", header = None)

    intrusion.columns = ['PID', 'SystemCalls']
    normal.columns = ['PID', 'SystemCalls']

    idict = {}
    ndict = {}

    for pid in list(set(intrusion['PID'].values)):
        idict[pid] = " ".join(str(value) for value in list(intrusion[intrusion['PID'] == pid]['SystemCalls'].values))

    for pid in list(set(normal['PID'].values)):
        ndict[pid] = " ".join(str(value) for value in list(normal[normal['PID'] == pid]['SystemCalls'].values))

    ilist = []
    nlist = []
    for key in idict.keys():
        s = idict[key]
        token = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(token, ngram)):
            ilist.append(ele)

    for key in ndict.keys():
        s = ndict[key]
        token = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(token, ngram)):
            nlist.append(ele)

    idf = pd.DataFrame(ilist)
    ndf = pd.DataFrame(nlist)

    idf['class'] = 1
    ndf['class'] = 0

    combined_df1 = pd.DataFrame()

    combined_df1 = combined_df1.append(idf)
    combined_df1 = combined_df1.append(ndf)

    print(combined_df1.shape)
    print()


    combined_df1 = combined_df1.drop_duplicates(subset = combined_df1.columns[:-1], keep = 'last')
    print(combined_df1.shape)
    print()

    from sklearn.metrics.pairwise import cosine_similarity
    import statistics

    similarity_list = []
    for i in range(0, 100):
        list1 = list(combined_df1[combined_df1['class'] == 0].sample(n = 1).iloc[0, :].values)
        list2 = list(combined_df1[combined_df1['class'] == 1].sample(n = 1).iloc[0, :].values)
        try:
            similarity_list.append(cosine_similarity([list1], [list2]))
        except Exception as e:
            print(e)
            print(list1, list2)

    print(statistics.median(similarity_list)[0][0])
    print()
    cosine_similar_ngram[ngram] = statistics.median(similarity_list)[0][0]
    

(8776, 6)

(168, 6)

0.7881717432175487

(8606, 11)

(213, 11)

0.730503465710548

(8436, 16)

(254, 16)

0.7149040650383824

(8266, 21)

(293, 21)

0.742060811012198

(8096, 26)

(333, 26)

0.7250925998279011



In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

cosine_similar_ngram_df = pd.DataFrame({"Ngram":list(cosine_similar_ngram.keys()), "Similarity_scores": list(cosine_similar_ngram.values())})

plt.rcParams['figure.figsize'] = [10, 7]

sns.lineplot(x = "Ngram", y = "Similarity_scores", data = cosine_similar_ngram_df)
    
plt.title("Line chart of the cosine similarity scores")
plt.xlabel("Ngram")
plt.ylabel("Cosine similarity scores")
plt.xticks(rotation = 90)
plt.show()